### Milestone 1 - Function Level Profiling

In [36]:
import cProfile,pstats
from mandelbort import compute_mandelbrot_naive, compute_mandelbrot_vectorized, benchmark

In [17]:
cProfile.run('compute_mandelbrot_naive(-2,1,-1.5,1.5,1024)','Naive_Profile.prof')
cProfile.run('compute_mandelbrot_vectorized(-2,1,-1.5,1.5,1024)','Numpy_Profile.prof')

for name in ('Naive_Profile.prof','Numpy_Profile.prof'):    
    stats = pstats.Stats(name)
    stats.sort_stats('cumulative')
    stats.print_stats(10)

Thu Feb 26 14:26:01 2026    Naive_Profile.prof

         23008342 function calls in 10.286 seconds

   Ordered by: cumulative time
   List reduced from 20 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   10.286   10.286 {built-in method builtins.exec}
        1    0.001    0.001   10.286   10.286 <string>:1(<module>)
        1    2.199    2.199   10.286   10.286 c:\Numerical_SC\mandelbrot-nsc\mandelbort.py:39(compute_mandelbrot_naive)
  1048576    6.814    0.000    8.086    0.000 c:\Numerical_SC\mandelbrot-nsc\mandelbort.py:59(mandelbrot_point_naive)
 21959734    1.273    0.000    1.273    0.000 {built-in method builtins.abs}
        2    0.000    0.000    0.000    0.000 c:\Users\rasm2\miniforge3\envs\nsc2026\Lib\site-packages\numpy\_core\function_base.py:26(linspace)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        2    0.000    0.000    0.000    0.000 {built-in metho

##### Function which takes the most total time ?
Naive mandel point naive function has total time of 6.806 s

For numpy cant see any times just 0.00000 s
##### Are there any functions called suprising amount of times ? 
The absolut function is being called 21959734 times while the naive mandel point function is called 1048576 times

##### How Does the Numpy profile compare to naive ? 

It cannot find the functions/read it, as the timings for functions are all zero for the numpy

So the only thing to compare is the total time of the function of 1.001 sec vs 10.267 sec

### Milestone 2 - Line-Level Profling

The results are shown above where it can be seen most of the time is indeed in the mandbrot point naive 83.6 % of the time

Here it can be seen that the absolut takes 40.8 % of the point naive, and then 38.9 % is in the z = z**2 + c

Thereby if 83.6% is this function then we can caluculate to get the total procent of abs and z calc things

In [25]:
print((83.6*(40.8+38.9))/100)

66.62919999999998


##### cProfile on naive vs NumPy: How many functions appear in each profile? What does this difference tell you about where the work actually happens?
23008342 functions for the naive

74 functions for the numpy

it tells me that me numpy optimize the function calls need for calculating the same thing

##### line profiler on naive: Which lines dominate runtime? What fraction of total time is spent in the inner loop?

If we talk about inner loop specifcally the for j in range(num):


In [ ]:
0.8 + 11.7 +83.6 +2.0 + 1.9 

100.0

The most dominiate is the 83.6% which is the naive point mandelbort specifically within this the most dominate is the abs and z=z**2 + c lines which has 40.8%  + 38.9 %

##### Based on your profiling results: why is NumPy faster than naive Python?
The functions are called less times and alot less time in total for the functinos

##### What would you need to change to make the naive version faster? (hint: what doesline profiler tell you about the inner loop?)

you would need to look at the abs and z=z**2 +c specifically i think yo look at making the operation one more then one thing at once, as it of now need to loop over and then call these operation alot of times

### Milestone 3 - Numba

In [50]:
from numba import njit
import numpy as np

@njit
def mandelbrot_point_numba(c):
    z = 0j
    max_iter = 100
    for n in range(max_iter):
        if z.real*z.real +  z.imag*z.imag > 4.0:
            return n 
        z = z**2 + c
    return max_iter 

@njit
def compute_mandelbrot_numba(x_min, x_max, y_min, y_max, num):
    x = np.linspace(x_min, x_max, num)   # real axis
    y = np.linspace(y_min, y_max, num)   # imaginary axis

    # 2D arrays to store results
    all_c = np.zeros((num, num), dtype=np.complex128)
    all_n = np.zeros((num, num), dtype=np.int64)

    for i in range(num):
        for j in range(num):
            c = x[i] + 1j * y[j]
            n = mandelbrot_point_numba(c)
            all_c[i, j] = c
            all_n[i, j] = n

    return all_n

def compute_mandelbrot_hybrid(x_min, x_max, y_min, y_max, num):
    x = np.linspace(x_min, x_max, num)   # real axis
    y = np.linspace(y_min, y_max, num)   # imaginary axis

    # 2D arrays to store results
    all_c = np.zeros((num, num), dtype=np.complex128)
    all_n = np.zeros((num, num), dtype=np.int64)

    for i in range(num):
        for j in range(num):
            c = x[i] + 1j * y[j]
            n = mandelbrot_point_numba(c)
            all_c[i, j] = c
            all_n[i, j] = n

    return all_n
    


#### Run timings for hybrid and fully numba

In [ ]:
_ = compute_mandelbrot_numba(-2 , 1, -1.5 , 1.5 , 1024) # warm -up
_ = compute_mandelbrot_hybrid(-2 , 1, -1.5 , 1.5 , 1024) # warm -up
median_time_numba, all_n_numba = benchmark(
        compute_mandelbrot_numba,
        -2, 1, -1.5, 1.5, 1024,
        n_runs=5, meta_prefix="Numba"
    )  
median_time_hybrid, all_n_hybrid = benchmark(
        compute_mandelbrot_hybrid,
        -2, 1, -1.5, 1.5, 1024,
        n_runs=5, meta_prefix="Numba Hybrid"
    )  
print (f" Hybrid : { median_time_hybrid:.3f}s")
print (f" Fully compiled : { median_time_numba:.3f}s")
print (f" Ratio : { median_time_hybrid / median_time_numba:.1f}x")


 Numba with Median:0.0528s ( min =0.0527, max =0.0533)
 Numba Hybrid with Median:2.0384s ( min =2.0007, max =2.0597)
 Hybrid : 2.038s
 Fully compiled : 0.053s
 Ratio : 38.6x
